Load model...

In [21]:
import keras

model = keras.models.load_model('model.h5')

Load vectorizer and extract model dimensions...

In [3]:
vectorizer = pickle.load(open('rationale-vectorizer.p'))

max_sentlen = vectorizer.maxlen
max_doclen = model.layers[1].input_length / max_sentlen # hacky way to get out `max_doclen`

Load a document...

In [69]:
%store -r doc_df

_, doc = next(iter(doc_df.groupby('doc_id')))

X_seq = vectorizer.texts_to_sequences(doc.sentence)
nb_sentence, _ = X_seq.shape
X_doc = np.zeros([max_doclen, max_sentlen])
X_doc[max_doclen-min(max_doclen, nb_sentence):] = X_seq[:max_doclen]

X = np.zeros([1, max_sentlen*max_doclen], dtype=np.int)
X[0] = X_doc.reshape([max_sentlen*max_doclen])

Predict sentence weights and document label...

In [78]:
import keras.backend as K

f = K.function(inputs=model.inputs+[K.learning_phase()],
               outputs=[model.get_layer('sent_weights').output, model.get_layer('doc_probs').output])

sent_weights, doc_probs = [output[0] for output in f([X, 1])] # only grab the result from the first document